<br>

# Introdução

In [1]:
import os
import pprint
import pandas as pd
from collections import OrderedDict

In [4]:
def get_parameters():
    # Read Data
    try:
        df_8468 = pd.read_excel(
            io=os.path.join(os.path.dirname(__file__), 'data', 'tab_dec_8468.xlsx'),
            sheet_name='dec_8468',
            index_col=0
        )
    except Exception as e:
        #print(e, '\n')
        #print('Read table from GitHub')
        df_8468 = pd.read_excel(
            io='https://raw.githubusercontent.com/gaemapiracicaba/norma_dec_8468-76/main/src/normas/data/tab_dec_8468.xlsx',
            sheet_name='dec_8468',
            index_col=0
        )

    # Filter only quality
    df_8468 = df_8468.loc[(df_8468['tipo_padrao'] == 'qualidade')]
    #print(df_8468.head())

    # Classes
    list_classes = list(set(df_8468['padrao_qualidade']))
    list_classes = [x for x in list_classes if pd.notnull(x)]
    list_classes.sort()    

    return df_8468, list_classes

In [6]:
df_8468, list_classes = get_parameters()
pprint.pprint(list_classes)

['Classe 1', 'Classe 2', 'Classe 3', 'Classe 4']


In [7]:
def filter_by_classe(df_8468, classe):
    # Filter dataframe by Classe
    df_8468 = df_8468.loc[(df_8468['padrao_qualidade'] == classe)]

    # Parâmetros
    list_parametros = list(set(df_8468['parametro_descricao']))
    list_parametros = [x for x in list_parametros if pd.notnull(x)]
    list_parametros.sort()    
    return df_8468, list_parametros

In [9]:
df_8468, list_parametros = filter_by_classe(df_8468, classe='Classe 2')
pprint.pprint(list_parametros)

['Amônia',
 'Arsênico',
 'Bário',
 'Chumbo',
 'Cianeto',
 'Cobre',
 'Coliformes',
 'Cromo Total',
 'Cádmio',
 'Demanda Bioquímica de Oxigênio',
 'Estanho',
 'Fenóis',
 'Flúor',
 'Mercúrio',
 'Nitrato',
 'Nitrito',
 'Oxigênio Dissolvido',
 'Selênio',
 'Zinco']


In [10]:
def filter_by_parameters(df_8468, parametro):
    # Filter dataframe by Parametro
    df_8468 = df_8468.loc[(df_8468['parametro_descricao'] == parametro)]

    # Check and Get Results
    if len(df_8468) == 1:
        dict_8468 = df_8468.to_dict(orient='records')[0]
        dict_8468 = OrderedDict(sorted(dict_8468.items(), key=lambda x: df_8468.columns.get_loc(x[0])))
        return dict_8468
    else:
        return 'erro'

In [11]:
# Filter Data by Parâmetros
dict_8468 = filter_by_parameters(df_8468, parametro='Oxigênio Dissolvido')
dict_8468

OrderedDict([('tipo_padrao', 'qualidade'),
             ('padrao_qualidade', 'Classe 2'),
             ('parametro_descricao', 'Oxigênio Dissolvido'),
             ('parametro_sigla', 'OD'),
             ('valor_minimo_permitido', 5.0),
             ('valor_maximo_permitido', nan),
             ('unidade', 'mg/l '),
             ('norma_referencia', 'Inciso V, Art. 11'),
             ('norma_texto',
              'Oxigênio Dissolvido (OD), em qualquer amostra, não inferior a 5 mg/l (cinco miligramas por litro)')])

In [12]:
def set_type_desconformidade(dict_8468):
    if pd.isnull(dict_8468['valor_minimo_permitido']) & pd.notnull(dict_8468['valor_maximo_permitido']):
        #print('Parâmetro só tem "valor máximo". Caso o valor medido esteja acima, é amostra desconforme!')
        tipo_8486 = 'acima>desconforme'

    elif pd.notnull(dict_8468['valor_minimo_permitido']) & pd.isnull(dict_8468['valor_maximo_permitido']):
        #print('Parâmetro só tem "valor mínimo". Caso o valor medido esteja abaixo, é amostra desconforme!')
        tipo_8486 = 'abaixo>desconforme'

    elif pd.notnull(dict_8468['valor_minimo_permitido']) & pd.notnull(dict_8468['valor_maximo_permitido']):
        #print('Parâmetro tem "valor mínimo" e "valor máximo". Caso o valor medido acima ou abaixo, é amostra desconforme!')
        tipo_8486 = 'abaixo_acima>desconforme'

    elif pd.isnull(dict_8468['valor_minimo_permitido']) & pd.isnull(dict_8468['valor_maximo_permitido']):
        #print('Erro!')
        tipo_8486 = 'erro'
    else:
        print('Erro!')
        #tipo_8486 = 'erro'

    return tipo_8486

In [13]:
set_type_desconformidade(dict_8468)

'abaixo>desconforme'

In [14]:
def evaluate_result(valor, dict_8468):
    # Get type
    tipo_8486 = set_type_desconformidade(dict_8468)

    # Evaluate type
    if tipo_8486 == 'acima>desconforme':
        if valor > dict_8468['valor_maximo_permitido']:
            result_8468 = 'desconforme'
        else:
            result_8468 = 'conforme'

    elif tipo_8486 == 'abaixo>desconforme':
        if valor < dict_8468['valor_minimo_permitido']:
            result_8468 = 'desconforme'
        else:
            result_8468 = 'conforme'

    elif tipo_8486 == 'abaixo_acima>desconforme':
        if dict_8468['valor_minimo_permitido'] <= valor <= dict_8468['valor_maximo_permitido']:
            result_8468 = 'conforme'
        else:
            result_8468 = 'desconforme'

    else:
        result_8468 = 'erro'

    return result_8468

In [15]:
valor = 10
evaluate_result(valor, dict_8468)

'conforme'

<br>

# Export

In [28]:
import os
from traitlets.config import Config
from nbconvert import PythonExporter
from nbconvert.preprocessors import TagRemovePreprocessor

In [29]:
input_filename = 'decreto_estadual_8468.ipynb'
input_filepath = os.path.join(os.getcwd(), input_filename)

In [30]:
# Import the exporter
c = Config()
c.TagRemovePreprocessor.enabled=True
c.ClearOutputPreprocessor.enabled=True
c.TemplateExporter.exclude_markdown=True
c.TemplateExporter.exclude_code_cell=False
c.TemplateExporter.exclude_input_prompt=True
c.TemplateExporter.exclude_output=True
c.TemplateExporter.exclude_raw=True
c.TagRemovePreprocessor.remove_cell_tags = ('remove_cell',)
c.TagRemovePreprocessor.remove_input_tags = ('remove_cell',)
c.TagRemovePreprocessor.remove_all_outputs_tags = ('remove_output',)
c.preprocessors = ['TagRemovePreprocessor']
c.PythonExporter.preprocessors = ['nbconvert.preprocessors.TagRemovePreprocessor']

# Configure and run out exporter
py_exporter = PythonExporter(config=c)
py_exporter.register_preprocessor(TagRemovePreprocessor(config=c), True)

# Configure and run out exporter - returns a tuple - first element with html, second with notebook metadata
body, metadata = PythonExporter(config=c).from_filename(input_filepath)

# Write to output html file
with open(os.path.join(os.getcwd(), '..', 'src', 'normas', 'decreto_estadual_8468.py'),  'w', encoding='utf-8') as f:
    f.write(body)